# Custom Hamiltonian for N=2 spatial orbital

In [1]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from block2 import SU2 as block2_SU2
from block2 import SZ as block2_SZ
from pathlib import Path
import planted_solutions.CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import planted_solutions.CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import planted_solutions.CAS.dmrghandler.src.dmrghandler as dmrghandler
from planted_solutions.CAS.dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from planted_solutions.DMRG_simulation.dmrg_simulation import get_ground_state_fci
import numpy as np
import planted_solutions.CAS_Cropping.saveload_utils as sl
import planted_solutions.CAS_Cropping.ferm_utils as feru
import planted_solutions.CAS_Cropping.var_utils as varu
import pyscf
import scipy
import openfermion as of
import planted_solutions.CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare

In [2]:
mol = 'h5'
Hf = sl.load_fermionic_hamiltonian(mol, prefix="../CAS/")
print(Hf)

FileNotFoundError: [Errno 2] No such file or directory: '../CAS/ham_lib/h5_fer.bin'

## OBT, TBT preparation

In [15]:
def custom_obt():
    """
    
    Returns:

    """
    tensor = np.zeros((2, 2))
    for p in range(2):
        for q in range(2):
            tensor[p ,q] = 1
    
    alternative = np.array([[-1.8351087041237648, 0, 0, 0], [0, -1.5506525118237189, 0, 0], [0, 0, -1.2458017459416912, 0], [0, 0, 0, -0.9063250085305642]])
    #alternative = np.array([[-1.8351087041237648, 0, 0, 0, 0], [0, -1.5506525118237189, 0, 0, 0], 
    #                         [0, 0, -1.2458017459416912, 0, 0], [0, 0, 0, -0.9063250085305642, 0], [0, 0, 0, 0, -0.9063250085305642]])
    #alternative = np.array([[-1.8351087041237648, 0, 0, 0, 0, 0, 0], [0, -1.5506525118237189, 0, 0, 0, 0, 0], 
    #                        [0, 0, -1.2458017459416912, 0, 0, 0, 0], [0, 0, 0, -0.9063250085305642, 0, 0, 0], [0, 0, 0, 0, -0.9063250085305642, 0, 0], 
    #                        [0, 0, 0, 0, 0, -1.2458017459416912, 0], [0, 0, 0, 0, 0, 0, -1.2458017459416912],])
    #alternative = np.array([[-1.11084418 , 0], [0, -0.589121]])
    #alternative = np.array(([-1.541249701637053 , 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]))
    #alternative = [np.array([[-1.541249701637053, 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]]),
    #              np.array([[-1.541249701637053, 0, 0], [0, -1.1554109573658455, 0], [0, 0, -0.744081101028083]])]
    # alternative = [
    #     np.array([[-1.11084418 , 0], [0, -0.589121]]),
    #     np.array([[-1.11084418 , 0], [0, -0.589121]])
    # ]
    return alternative


def custom_tbt():
    """
    
    Returns:

    """
    n = 4
    sep1 = 2
    sep2 = 4
    sep3 = 4
    tensor = np.zeros((n, n, n, n))
    # for p in range(sep1):
    #     for q in range(sep1):
    #         for r in range(sep1):
    #             for s in range(sep1): 
    #                 tensor[p ,q, r, s] = 0.01
    #                 # if p == q and r == s:
    #                 #     tensor[p, q, r, s] = 0.1
    #                 # elif p == r and q == s:
    #                 #     tensor[p, q, r, s] = 0.1
    #                 # elif p == s and r == q:
    #                 #     tensor[p, q, r, s] = 0.1
    #                 # else:
    #                 #     tensor[p, q, r, s] = 0
    
    set1 = [0, 1]
    set2 = [2, 3]
    for p in set1:
        for q in set1:
            for r in set1:
                for s in set1:
                    tensor[p, q, r, s] = 0.1
    
    for p in set2:
        for q in set2:
            for r in set2:
                for s in set2:
                    tensor[p, q, r, s] = 0.1
        
    
    for p in range(sep1, sep2):
        for q in range(sep1, sep2):
            for r in range(sep1, sep2):
                for s in range(sep1, sep2):
                    tensor[p ,q, r, s] = 0.01
    
    for p in range(sep2, sep3):
        for q in range(sep2, sep3):
            for r in range(sep2, sep3):
                for s in range(sep2, sep3):
                    tensor[p ,q, r, s] = 0.01
                    
    #sz_tensor = [tensor, tensor,  tensor]
    # for p in range(3, 7):
    #     for q in range(3, 7):
    #         for r in range(3, 7):
    #             for s in range(3, 7):
    #                 tensor[p ,q, r, s] = 0.01
    # 
    # 
    # for p in range(5, 7):
    #     for q in range(5, 7):
    #         for r in range(5, 7):
    #             for s in range(5, 7):
    #                 tensor[p ,q, r, s] = 0.01
    
    # tensor[2, 1, 0, 1] = 0.1
    # tensor[1, 2, 1, 0] = 0.1
    # tensor[2, 1, 1, 0] = 0.1
    # tensor[1, 2, 0, 1] = 0.1
    # 
    # tensor[2, 2, 0, 1] = 0.1
    # tensor[2, 2, 1, 0] = 0.1
    # 
    # tensor[2, 1, 1, 1] = 0.1
    # tensor[1, 2, 1, 1] = 0.1
    
    
    # |1100> to |1001> or |0110>
    # tensor[1, 0, 0, 0] = 0
    # tensor[0, 1, 0, 0] = 0
    # tensor[0, 0, 1, 0] = 0
    # tensor[0, 0, 0, 1] = 0
    # |0011> to |1001> or |0110>
    # tensor[1, 1, 1, 0] = 0
    # tensor[1, 1, 0, 1] = 0
    # tensor[1, 0, 1, 1] = 0
    # tensor[0, 1, 1, 1] = 0  
    
    return tensor

def orbital_reorder():
    return np.array([[1, 0], [0, -1]])

def connection_strength(symmetry, tbt):
    commutator = np.matmul(tbt, symmetry) - np.matmul(symmetry, tbt)
    return scipy.linalg.norm(commutator, ord=None)

In [16]:
def mf_hamiltonian_obt():
    tensor = np.zeros((2, 2))
    for p in range(2):
        for q in range(2):
            tensor[p ,q] = 1
    alternative = np.array([[-1.8351087041237648, 0, 0, 0], [0, -1.5506525118237189, 0, 0], [0, 0, -1.2458017459416912, 0], [0, 0, 0, -0.9063250085305642]])
    return alternative

def mf_hamiltonian_tbt():
    n = 4
    tensor = np.zeros((n, n, n, n))
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    if p == q and r == s:
                        tensor[p, q, r, s] = 1
    zero_tensor = np.zeros((n, n, n, n))
    return [zero_tensor, tensor, zero_tensor]

In [7]:
(
one_body_tensor,
two_body_tensor,
nuc_rep_energy,
num_orbitals,
num_spin_orbitals,
num_electrons,
two_S,
two_Sz,
orb_sym,
extra_attributes,
) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path("custom_fcidumps")/Path("fcidump.H2O_21_original"), molpro_orbsym_convention=True)

Parsing custom_fcidumps/fcidump.H2O_21_original


## DMRG simulation

In [8]:
print(num_electrons)
print(two_Sz)

10
0


In [9]:
from planted_solutions.DMRG_simulation.format_dmrg.format_dmrg_param import get_dmrg_param, get_dmrg_process_param


default_final_bond_dim = 1
default_sweep_schedule_bond_dims = [default_final_bond_dim] * 4 + [
    default_final_bond_dim + 3
] * 4
default_sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
default_sweep_schedule_davidson_threshold = [1e-20] * 8

init_state_bond_dimension = 10
max_num_sweeps = 200
energy_convergence_threshold = 1e-8
sweep_schedule_bond_dims = default_sweep_schedule_bond_dims
sweep_schedule_noise = default_sweep_schedule_noise
sweep_schedule_davidson_threshold = (
    default_sweep_schedule_davidson_threshold
)
# nuc_rep_energy = 0
#     
# num_orbitals = 4
# num_electrons = 4
num_unpaired = 0
multiplicity = 1

dmrg_process_param = get_dmrg_process_param(
    init_state_bond_dimension, max_num_sweeps, energy_convergence_threshold,
    sweep_schedule_bond_dims, sweep_schedule_noise,
    sweep_schedule_davidson_threshold)
dmrg_param = get_dmrg_param(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity, dmrg_process_param)

result = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(one_body_tensor, two_body_tensor, dmrg_param)
mps = result['ket_optimized']
print(result)

dtrie finished 0.012530875000010155
Number of DET =        441 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 2002222  =    0.262263937103835
DET          1 0ab2222  =    0.250526606870740
DET          2 2202ab2  =   -0.249063997889014
DET          3 0202222  =    0.219985466681612
DET          4 0022222  =   -0.216577514524135
DET          5 22b20a2  =   -0.204443217833373
DET          6 a0b2222  =    0.201708839639433
DET          7 220ba22  =    0.177186028164017
DET          8 22baab2  =   -0.166737743557271
DET          9 22b0a22  =    0.156130704924799
DET         10 220a22b  =    0.154371637876376
DET         11 220a2b2  =   -0.141753903522058
DET         12 220b2a2  =   -0.138614510927236
DET         13 22b2aab  =   -0.129522546274940
DET         14 2022ab2  =   -0.128884743014084
DET         15 0ba2222  =    0.126809960500893
DET         16 22b02a2  =   -0.122142708023469
DET         17 0222ab2  =   -0.108107773616117
DET         18 

In [10]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")

Deleted './mps_directory' and all its contents.


In [11]:
loop_results = dmrg_central_loop(one_body_tensor, two_body_tensor, dmrg_param, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       1428

Build MPO | Nsites =     7 | Nterms =       1428 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.004
 Site =     1 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =      155 SPT = 0.8972 Tmvc = 0.000 T = 0.004
 Site =     2 /     7 .. Mmpo =   110 DW = 0.00e+00 NNZ =      259 SPT = 0.9594 Tmvc = 0.000 T = 0.004
 Site =     3 /     7 .. Mmpo =    78 DW = 0.00e+00 NNZ =      721 SPT = 0.9160 Tmvc = 0.001 T = 0.005
 Site =     4 /     7 .. Mmpo =    66 DW = 0.00e+00 NNZ =      139 SPT = 0.9730 Tmvc = 0.000 T = 0.004
 Site =     5 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =      143 SPT = 0.9015 Tmvc = 0.000 T = 0.003
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.002
Ttotal =      0.027 Tmvc-total = 0.002 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         1465 SIZE =        23116 SPT = 0.9366

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1428

Build MPO | Nsites =     7 | Nterms =       1428 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =      155 SPT = 0.8972 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =   110 DW = 0.00e+00 NNZ =      259 SPT = 0.9594 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    78 DW = 0.00e+00 NNZ =      721 SPT = 0.9160 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    66 DW = 0.00e+00 NNZ =      139 SPT = 0.9730 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =      143 SPT = 0.9015 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.008 Tmvc-total = 0.001 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         1465 SIZE =        23116 SPT = 0.9366

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1428

Build MPO | Nsites =     7 | Nterms =       1428 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =      155 SPT = 0.8972 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =   110 DW = 0.00e+00 NNZ =      259 SPT = 0.9594 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =    78 DW = 0.00e+00 NNZ =      721 SPT = 0.9160 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    66 DW = 0.00e+00 NNZ =      139 SPT = 0.9730 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =      143 SPT = 0.9015 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.009 Tmvc-total = 0.001 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         1465 SIZE =        23116 SPT = 0.9366

...

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1428

Build MPO | Nsites =     7 | Nterms =       1428 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.003
 Site =     1 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =      155 SPT = 0.8972 Tmvc = 0.000 T = 0.004
 Site =     2 /     7 .. Mmpo =   110 DW = 0.00e+00 NNZ =      259 SPT = 0.9594 Tmvc = 0.000 T = 0.005
 Site =     3 /     7 .. Mmpo =    78 DW = 0.00e+00 NNZ =      721 SPT = 0.9160 Tmvc = 0.000 T = 0.006
 Site =     4 /     7 .. Mmpo =    66 DW = 0.00e+00 NNZ =      139 SPT = 0.9730 Tmvc = 0.000 T = 0.003
 Site =     5 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =      143 SPT = 0.9015 Tmvc = 0.000 T = 0.003
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.004
Ttotal =      0.029 Tmvc-total = 0.002 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         1465 SIZE =        23116 SPT = 0.9366

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1428

Build MPO | Nsites =     7 | Nterms =       1428 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =      155 SPT = 0.8972 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   110 DW = 0.00e+00 NNZ =      259 SPT = 0.9594 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =    78 DW = 0.00e+00 NNZ =      721 SPT = 0.9160 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    66 DW = 0.00e+00 NNZ =      139 SPT = 0.9730 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =      143 SPT = 0.9015 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.011 Tmvc-total = 0.001 MPO bond dimension =   110 MaxDW = 0.00e+00
NNZ =         1465 SIZE =        23116 SPT = 0.9366

/Users/rick/planted_solutions/planted_solutions/CAS/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


In [9]:
print(loop_results["final_dmrg_results"])

{'dmrg_ground_state_energy': -6.738128088866974, 'initial_bond_dims': 15, 'dmrg_discarded_weight': 0.0, 'sweep_bond_dims': array([22, 22, 22, 22, 44, 44, 44, 44, 44]), 'sweep_max_discarded_weight': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'sweep_energies': array([[-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809],
       [-6.73812809]]), 'wall_make_driver_time_s': 9.5375e-05, 'cpu_make_driver_time_s': 9.5e-05, 'wall_driver_initialize_system_time_s': 0.000174792, 'cpu_driver_initialize_system_time_s': 0.000175, 'wall_reorder_integrals_time_s': 2.666e-06, 'cpu_reorder_integrals_time_s': 3e-06, 'wall_get_qchem_hami_mpo_time_s': 0.006884667, 'cpu_get_qchem_hami_mpo_time_s': 0.007304, 'wall_generate_initial_mps_time_s': 0.002178458, 'cpu_generate_initial_mps_time_s': 0.001869, 'wall_copy_mps_time_s': 0.001882833, 'cpu_copy_mps_time_s': 0.001688, 'wall_dmrg_optimization_ti

In [10]:
mps_optimized = loop_results["final_dmrg_results"]["ket_optimized"]

KeyError: 'ket_optimized'

In [9]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")

In [10]:
mps_dir = Path("./mps_directory/mps_storage/first_preloop_calc_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)

In [11]:
tools.init(block2_SZ)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)

cp: mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/first_preloop_calc_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storag

In [12]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)

In [13]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)

In [ ]:
ener = driver.expectation(mps, mpo, mps)
print(ener)

bra = driver.get_random_mps(tag="GS5", bond_dim=250, nroots=1)
applied_mps = driver.multiply(bra ,mpo, mps)
print(applied_mps)

variance = np.sqrt(abs(applied_mps ** 2 - ener ** 2))
print(f"Variance: {variance}")
print(f"Variance ratio: {variance / ener}")

In [13]:
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)

mps center changed (temporarily)
dtrie finished 0.0026265830000014034
Number of DET =          1 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 2ab0  =    1.000000000000000


In [43]:
zket = driver.mps_change_to_sz(mps, "ZKET", num_unpaired)

driver.symm_type = SymmetryTypes.SZ
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)
csfs, vals = driver.get_csf_coefficients(zket, cutoff=1e-30, iprint=1)

mps center changed (temporarily)
dtrie finished 0.00167262499996923
Number of DET =          1 (cutoff =     1e-30)
Sum of weights of included DET =    0.333333333333333

DET          0 22a000a  =   -0.577350269189626


# Initial state 

In [78]:
driver.initialize_system(
        n_sites=num_orbitals,
        n_elec=num_electrons,
        spin=num_unpaired,
        orb_sym=None,
        heis_twos=-1,  # Default value
        heis_twosz=0,  # Default value
        singlet_embedding=True,  # Default value
        pauli_mode=False,  # Default value
        vacuum=None,  # Default value
        left_vacuum=None,  # Default value
        target=None,  # Default value
        hamil_init=True,  # Default value
    )

In [79]:
initial_ket = driver.get_random_mps(
        tag="init_ket",
        bond_dim=init_state_bond_dimension,
        center=0,  # Default value, canonical center of MPS
        dot=2,  # Default value, site type of MPS
        target=None,  # Default value, target quantum number
        nroots=1,  # Default value, number of roots, use 1 for ground state
        occs=None,  # Hint of occupancy information, if None, uniform distribution is assumed
        full_fci=True,  # If True, the full FCI space is used, default is True
        left_vacuum=None,  # Default value, only has effects for SU(2) and SE MPS with non-single target
        casci_ncore=0,  # For CASCI MPS, default is 0
        casci_nvirt=0,  # For CASCI MPS, default is 0
        mrci_order=0,  # For CASCI MPS, default is 0
        orig_dot=False,  # Default is False; if False, create MPS as one-site, then convert to two-site
        # If True, create MPS as two-site or one-site directly
    )

In [80]:
print(initial_ket)

In [81]:
csfs, coeffs = driver.get_csf_coefficients(initial_ket, cutoff=1e-308, iprint=1)

dtrie finished 0.0010554160000992852
Number of CSF =         20 (cutoff =    1e-308)
Sum of weights of included CSF =    1.000000000000000

CSF          0 +-+-  =    0.511647419102260
CSF          1 +02-  =   -0.345405259682904
CSF          2 0202  =    0.312652689074099
CSF          3 ++--  =    0.302903609814349
CSF          4 +0-2  =    0.296559889518087
CSF          5 +-20  =   -0.249657018604107
CSF          6 +2-0  =    0.244858194353664
CSF          7 2+0-  =    0.230573886712273
CSF          8 0+2-  =   -0.209190907603900
CSF          9 2+-0  =    0.198043941111994
CSF         10 20+-  =    0.147033182371957
CSF         11 02+-  =    0.144812348959025
CSF         12 0022  =   -0.122765514749004
CSF         13 0220  =   -0.103871290210870
CSF         14 0+-2  =    0.076864965231922
CSF         15 +20-  =   -0.066147070632627
CSF         16 +-02  =    0.056996280023964
CSF         17 2020  =   -0.025907251911987
CSF         18 2002  =   -0.017434429801974
CSF         19 2200  =  